# Spectral analyses

## Introductory notes:
This notebook presents spectral analyses functionality:
* Power spectral density (PSD) per sleep stage
* Spectrogram & hypnogram
* Topomaps for the spectra per sleep stage
* Spectral parametrization with FOOOF
* Additional results such as sleep statistics.

Recommended watching and reading:
1. [Mike X Cohen: Static spectral analysis](https://youtube.com/playlist?list=PLn0OLiymPak2jxGCbWrcgmXUtt9Lbjj_A)
2. [MNE: The Spectrum and EpochsSpectrum classes: frequency-domain data](https://mne.tools/stable/auto_tutorials/time-freq/10_spectrum_class.html#)
3. [FOOOF: Tutorials](https://fooof-tools.github.io/fooof/auto_tutorials/index.html)
4. [MNE: SpectrumArray class](https://mne.tools/stable/generated/mne.time_frequency.SpectrumArray.html)

## Import data

### Import module

In [ ]:
from sleepeegpy.pipeline import SpectralPipe
from os import makedirs, path

By default, all the input files are assumed to be saved in <b>input_files</b>, which will be created (if not already exists) in the notebook path.
Change the following string to use another path

In [ ]:
output_dir = "output_folder"  # Output path and name can be changed here
input_dir = "input_files"  # input files dir can be changed here
makedirs(input_dir, exist_ok=True)
makedirs(output_dir, exist_ok=True)

#### Add required files
* Put all your files in the input folder.
* Modify your eeg file name below. The file can be any format supported by the mne.read_raw() function.
* Modify your hypnogram file name (Point-per-row type of hypnogram) below.
* If needed, change Hypnogram's sampling frequency 
* For more information about the supported formats, see [mne documentation](https://mne.tools/stable/generated/mne.io.Raw.html)

In [ ]:
eeg_file_name = "resampled_raw.fif"  # add your eeg_path here
hypnogram_filename = "staging.txt"  # Point-per-row type of hypnogram.
hypno_freq = 1  # If required, change Hypnogram's sampling frequency (visbrain's hypnograms default to 1)

### Initialize SpectralPipe object

In [ ]:
path_to_eeg = path.join(input_dir, eeg_file_name)
hypnogram_path = path.join(input_dir, hypnogram_filename)
assert path.isfile(path_to_eeg) and path.isfile(
    hypnogram_path
), f"{path_to_eeg} or {hypnogram_path} not exist"

spectral_pipe = SpectralPipe(
    path_to_eeg=path_to_eeg,
    output_dir=output_dir,
    path_to_hypno=hypnogram_path,
    hypno_freq=hypno_freq,
)

## Compute PSD

In [ ]:
spectral_pipe.compute_psd(
    # A dict describing stages and their indices in the hypnogram file.
    sleep_stages={"Wake": 0, "N1": 1, "N2": 2, "N3": 3, "REM": 4},
    # Rereferencing to apply. Can be list of str channels or "average".
    # If None, will not change the reference.
    reference="average",
    fmin=0,  # Lower frequency bound.
    fmax=60,  # Upper frequency bound.
    picks="eeg",  # Channels to compute the PSD for.
    reject_by_annotation=True,  # Whether to reject epochs annotated as BAD.
    save=True,  # Whether to save the PSD hdf5 file for each sleep stage.
    overwrite=True,  # Whether to overwrite hdf5 files if there are any.
    # Additional arguments passed to the PSD computing method, i.e., welch or multitaper:
    n_fft=1024,
    n_per_seg=1024,
    n_overlap=512,
    window="hamming",
    n_jobs=-1,
    verbose=False,
)

In [ ]:
spectral_pipe.psds["REM"].get_data()

In [ ]:
spectral_pipe.psds["REM"].to_data_frame()

## Visualize

### PSD

In [ ]:
spectral_pipe.plot_psds(
    picks=["E101"],
    psd_range=(-20, 30),  # Y axis limits
    freq_range=(0, 40),  # X axis limits
    dB=True,
    xscale="linear",  # Matplotlib xscale. Can be {"linear", "log", "symlog", "logit", ...} or ScaleBase
    axis=None,
    plot_sensors=True,  # Whether to plot EEG sensors showing which channels were used to compute PSD.
    save=True,  # Whether to save the plot as a png file.
)

In [ ]:
_ = spectral_pipe.psds["N2"].plot(picks="data", exclude="bads", show=False)

### Hypnogram & spectrogram

In [ ]:
spectral_pipe.plot_hypnospectrogram(
    picks=["E101"],  # Channel[s] to compute the spectrogram on.
    win_sec=10,  # The length of the sliding window, in seconds, used for multitaper PSD computation.
    freq_range=(0, 40),  # Y axis limits
    cmap="Spectral_r",  # Matplotlib colormap as in https://matplotlib.org/stable/tutorials/colors/colormaps.html
    overlap=True,  # Whether to plot hypnogram over spectrogram (True) or on top of it (False)
    save=True,  # Whether to save the plot as a file.
)

### Topomap

Plots a topomap for a single sleep stage and frequency band

In [ ]:
spectral_pipe.plot_topomap(
    stage="N2",  # Stage to plot topomap for.
    band={"SMR": (12.5, 15)},  # Band to plot topomap for.
    # Should contain at least index of the provided "stage".
    dB=False,  # Whether to transform PSD to dB/Hz
    axis=None,  # Whether to plot on provided matplotlib axis.
    save=True,  # Whether to save the plot as a file.
    topomap_args=dict(cmap="plasma"),  # Arguments passed to mne.viz.plot_topomap().
    cbar_args=None,  # Arguments passed to plt.colorbar().
)

### Topomap collage
Plot topomaps for multiple bands and sleep stages

In [ ]:
spectral_pipe.plot_topomap_collage(
    #  Bands to plot topomaps for.
    bands={
        "Delta": (0, 4),
        "Theta": (4, 8),
        "Alpha": (8, 12.5),
        "SMR": (12.5, 15),
        "Beta": (12.5, 30),
        "Gamma": (30, 60),
    },
    # Tuple of strs or "all", e.g., ("N1", "REM") or "all" (plots all "sleep_stages").
    stages_to_plot="all",
    dB=False,  # Whether to transform PSD to dB/Hz.
    low_percentile=5,  # Set min color value by percentile of the band data.
    high_percentile=95,  # Set max color value by percentile of the band data.
    fig=None,  # Instance of plt.Figure, a new fig will be created if None.
    save=True,  # Whether to save the plot as a file.
    topomap_args=dict(cmap="plasma"),  # Arguments passed to mne.viz.plot_topomap().
    cbar_args=None,  # Arguments passed to plt.colorbar().
)

## Parametrize spectrum

In [ ]:
spectral_pipe.parametrize(
    picks=["eeg"],  # Channels to use.
    freq_range=[0.5, 60],  # Range of frequencies to parametrize.
    # Whether to average psds over channels.
    # If False or multiple channels are provided, the FOOOFGroup will be used.
    # Defaults to False.
    average_ch=False,
)

In [ ]:
spectral_pipe.fooofs["N2"].report()

## Sleep Stats

In [ ]:
spectral_pipe.sleep_stats(save=False)